In [19]:
import tensorflow as tf
config = tf.ConfigProto(allow_soft_placement=True) 
config.gpu_options.allow_growth = True
sess = tf.Session(config=config)
sess = tf.Session(config=tf.ConfigProto(log_device_placement=True))

In [20]:
import numpy as np
import keras.backend as K
from keras.layers import Input, Lambda
from keras.models import Model
from keras.optimizers import Adam
from keras.callbacks import TensorBoard, ModelCheckpoint, ReduceLROnPlateau, EarlyStopping

from yolo3.model import preprocess_true_boxes, yolo_body, tiny_yolo_body, yolo_loss
from yolo3.utils import get_random_data

In [21]:
def _main():
    annotation_path = '2007_train.txt' #待訓練清單(YOLO格式）
    log_dir = 'logs/000/' #訓練過程及結果暫存路徑
    classes_path = 'model_data/my_classes.txt' #自定義標籤檔路徑及名稱
    anchors_path = 'model_data/yolo_anchors.txt' #錨點定義檔路徑及名稱
    class_names = get_classes(classes_path)
    num_classes = len(class_names)
    anchors = get_anchors(anchors_path)

    input_shape = (416,416) # multiple of 32, hw 預設輸入影像尺寸須為32的倍數(寬，高)

    is_tiny_version = len(anchors)==6 # default setting
    if is_tiny_version:
        model = create_tiny_model(input_shape, anchors, num_classes,
            freeze_body=2, weights_path='model_data/tiny_yolo_weights.h5')
    else:
        model = create_model(input_shape, anchors, num_classes,
            freeze_body=2, weights_path='model_data/yolo_weights.h5') #指定起始訓練權重檔路徑及名稱
        
    logging = TensorBoard(log_dir=log_dir)
    checkpoint = ModelCheckpoint(log_dir + 'ep{epoch:03d}-loss{loss:.3f}-val_loss{val_loss:.3f}.h5',
        monitor='val_loss', save_weights_only=True, save_best_only=True, period=3) #訓練過程權重檔名稱由第幾輪加上損失率為名稱
    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=3, verbose=1)
    early_stopping = EarlyStopping(monitor='val_loss', min_delta=0, patience=10, verbose=1,mode='auto')

    val_split = 0.1
    with open(annotation_path) as f:
        lines = f.readlines()
    np.random.seed(10101)
    np.random.shuffle(lines)
    np.random.seed(None)
    num_val = int(len(lines)*val_split)
    num_train = len(lines) - num_val

    # Train with frozen layers first, to get a stable loss.
    # Adjust num epochs to your dataset. This step is enough to obtain a not bad model.
    if True:
        model.compile(optimizer=Adam(lr=1e-3), loss={
            # use custom yolo_loss Lambda layer.
            'yolo_loss': lambda y_true, y_pred: y_pred})

        batch_size = 1 #批次處理數量，依GPU記憶體大小決定
        print('Train on {} samples, val on {} samples, with batch size {}.'.format(num_train, num_val, batch_size))
        model.fit_generator(data_generator_wrapper(lines[:num_train], batch_size, input_shape, anchors, num_classes),
                steps_per_epoch=max(1, num_train//batch_size),
                validation_data=data_generator_wrapper(lines[num_train:], batch_size, input_shape, anchors, num_classes),
                validation_steps=max(1, num_val//batch_size),
                epochs=50, #訓練遍歷次數
                initial_epoch=0, #初始訓練遍歷次數
                callbacks=[logging, checkpoint])
    
        model.save_weights(log_dir + 'trained_weights_stage_1.h5') #儲存臨時權重檔案名稱

    # 解凍並繼續訓練以進行微調
    # 如果效果不好則訓練更長時間
    if True:
        for i in range(len(model.layers)):
            model.layers[i].trainable = True
        model.compile(optimizer=Adam(lr=1e-4), loss={'yolo_loss': lambda y_true, y_pred: y_pred}) # recompile to apply the change
        print('Unfreeze all of the layers.')

        batch_size = 1 # note that more GPU memory is required after unfreezing the body
        print('Train on {} samples, val on {} samples, with batch size {}.'.format(num_train, num_val, batch_size))
        model.fit_generator(data_generator_wrapper(lines[:num_train], batch_size, input_shape, anchors, num_classes),
            steps_per_epoch=max(1, num_train//batch_size),
            validation_data=data_generator_wrapper(lines[num_train:], batch_size, input_shape, anchors, num_classes),
            validation_steps=max(1, num_val//batch_size),
            epochs=100, #訓練遍歷次數
            initial_epoch=50, #初始訓練遍歷次數
            callbacks=[logging, checkpoint, reduce_lr, early_stopping])
        
        model.save_weights(log_dir + 'trained_weights_final.h5') #儲存最終權重檔
        #model.save(log_dir + 'trained_model_final.h5') #儲存完整模型及權重檔

    # Further training if needed.

In [22]:
def get_classes(classes_path):
    '''loads the classes'''
    with open(classes_path) as f:
        class_names = f.readlines()
    class_names = [c.strip() for c in class_names]
    return class_names

In [23]:
def get_anchors(anchors_path):
    '''loads the anchors from a file'''
    with open(anchors_path) as f:
        anchors = f.readline()
    anchors = [float(x) for x in anchors.split(',')]
    return np.array(anchors).reshape(-1, 2)

In [24]:
def create_model(input_shape, anchors, num_classes, load_pretrained=True, freeze_body=2,
            weights_path='model_data/yolo_weights.h5'):
    '''create the training model'''
    K.clear_session() # get a new session
    image_input = Input(shape=(None, None, 3))
    h, w = input_shape
    num_anchors = len(anchors)

    y_true = [Input(shape=(h//{0:32, 1:16, 2:8}[l], w//{0:32, 1:16, 2:8}[l], \
        num_anchors//3, num_classes+5)) for l in range(3)]

    model_body = yolo_body(image_input, num_anchors//3, num_classes)
    print('Create YOLOv3 model with {} anchors and {} classes.'.format(num_anchors, num_classes))

    if load_pretrained:
        model_body.load_weights(weights_path, by_name=True, skip_mismatch=True)
        print('Load weights {}.'.format(weights_path))
        if freeze_body in [1, 2]:
            # Freeze darknet53 body or freeze all but 3 output layers.
            num = (185, len(model_body.layers)-3)[freeze_body-1]
            for i in range(num): model_body.layers[i].trainable = False
            print('Freeze the first {} layers of total {} layers.'.format(num, len(model_body.layers)))

    model_loss = Lambda(yolo_loss, output_shape=(1,), name='yolo_loss',
        arguments={'anchors': anchors, 'num_classes': num_classes, 'ignore_thresh': 0.5})(
        [*model_body.output, *y_true])
    model = Model([model_body.input, *y_true], model_loss)

    return model

In [25]:
def create_tiny_model(input_shape, anchors, num_classes, load_pretrained=True, freeze_body=2,
            weights_path='model_data/tiny_yolo_weights.h5'):
    '''create the training model, for Tiny YOLOv3'''
    K.clear_session() # get a new session
    image_input = Input(shape=(None, None, 3))
    h, w = input_shape
    num_anchors = len(anchors)

    y_true = [Input(shape=(h//{0:32, 1:16}[l], w//{0:32, 1:16}[l], \
        num_anchors//2, num_classes+5)) for l in range(2)]

    model_body = tiny_yolo_body(image_input, num_anchors//2, num_classes)
    print('Create Tiny YOLOv3 model with {} anchors and {} classes.'.format(num_anchors, num_classes))

    if load_pretrained:
        model_body.load_weights(weights_path, by_name=True, skip_mismatch=True)
        print('Load weights {}.'.format(weights_path))
        if freeze_body in [1, 2]:
            # Freeze the darknet body or freeze all but 2 output layers.
            num = (20, len(model_body.layers)-2)[freeze_body-1]
            for i in range(num): model_body.layers[i].trainable = False
            print('Freeze the first {} layers of total {} layers.'.format(num, len(model_body.layers)))

    model_loss = Lambda(yolo_loss, output_shape=(1,), name='yolo_loss',
        arguments={'anchors': anchors, 'num_classes': num_classes, 'ignore_thresh': 0.7})(
        [*model_body.output, *y_true])
    model = Model([model_body.input, *y_true], model_loss)

    return model

In [26]:
def data_generator(annotation_lines, batch_size, input_shape, anchors, num_classes):
    '''data generator for fit_generator'''
    n = len(annotation_lines)
    i = 0
    while True:
        image_data = []
        box_data = []
        for b in range(batch_size):
            if i==0:
                np.random.shuffle(annotation_lines)
            image, box = get_random_data(annotation_lines[i], input_shape, random=True)
            image_data.append(image)
            box_data.append(box)
            i = (i+1) % n
        image_data = np.array(image_data)
        box_data = np.array(box_data)
        y_true = preprocess_true_boxes(box_data, input_shape, anchors, num_classes)
        yield [image_data, *y_true], np.zeros(batch_size)

In [27]:
def data_generator_wrapper(annotation_lines, batch_size, input_shape, anchors, num_classes):
    n = len(annotation_lines)
    if n==0 or batch_size<=0: return None
    return data_generator(annotation_lines, batch_size, input_shape, anchors, num_classes)

In [28]:
if __name__ == '__main__':
    _main()

Create YOLOv3 model with 9 anchors and 2 classes.
Load weights model_data/yolo_weights.h5.
Freeze the first 249 layers of total 252 layers.
Train on 108 samples, val on 12 samples, with batch size 1.
Epoch 1/50
108/108 [==============================] - 18s 170ms/step - loss: 1132.8628 - val_loss: 109.8321
Epoch 2/50
108/108 [==============================] - 16s 148ms/step - loss: 87.5034 - val_loss: 56.6417
Epoch 3/50
108/108 [==============================] - 14s 132ms/step - loss: 54.3917 - val_loss: 42.1694
Epoch 4/50
108/108 [==============================] - 15s 140ms/step - loss: 42.0986 - val_loss: 34.3749
Epoch 5/50
108/108 [==============================] - 15s 142ms/step - loss: 35.6892 - val_loss: 28.0455
Epoch 6/50
108/108 [==============================] - 18s 163ms/step - loss: 31.0454 - val_loss: 28.2281
Epoch 7/50
108/108 [==============================] - 15s 138ms/step - loss: 28.2883 - val_loss: 25.6547
Epoch 8/50
108/108 [==============================] - 14s 128m

108/108 [==============================] - 52s 483ms/step - loss: 14.2324 - val_loss: 16.5699
Epoch 73/100
108/108 [==============================] - 52s 483ms/step - loss: 14.2024 - val_loss: 14.5993
Epoch 74/100
108/108 [==============================] - 53s 486ms/step - loss: 14.8793 - val_loss: 14.8592

Epoch 00074: ReduceLROnPlateau reducing learning rate to 9.999999939225292e-10.
Epoch 75/100
108/108 [==============================] - 52s 484ms/step - loss: 14.2471 - val_loss: 16.0455
Epoch 76/100
108/108 [==============================] - 52s 485ms/step - loss: 14.1447 - val_loss: 17.1170
Epoch 77/100
108/108 [==============================] - 52s 483ms/step - loss: 14.3349 - val_loss: 16.4247

Epoch 00077: ReduceLROnPlateau reducing learning rate to 9.999999717180686e-11.
Epoch 78/100
108/108 [==============================] - 53s 486ms/step - loss: 14.2876 - val_loss: 15.7932
Epoch 79/100
108/108 [==============================] - 53s 489ms/step - loss: 14.5898 - val_loss: 15.